In [1]:
import numpy as np
import numpy.typing as npt
import pytest
import stim
from graphix import Circuit, Pattern, command
from graphix.fundamentals import Plane
from graphix.noise_models.depolarising_noise_model import DepolarisingNoiseModel
from graphix.noise_models.noise_model import NoiseModel
from graphix.random_objects import rand_circuit
from graphix.sim.base_backend import (
    FixedBranchSelector,
    RandomBranchSelector,
)
from graphix.simulator import DefaultMeasureMethod
from graphix.states import BasicStates
from numpy.random import PCG64, Generator
from veriphix.client import Client, Secrets
from veriphix.trappifiedCanvas import TrappifiedCanvas

from gospel.brickwork_state_transpiler import (
    ConstructionOrder,
    generate_random_pauli_pattern,
    get_bipartite_coloring,
)
from gospel.scripts import compare_backend_results
from gospel.stim_pauli_preprocessing import (
    StimBackend,
    cut_pattern,
    preprocess_pauli,
    simulate_pauli,
)

/Users/amsaha/gospel/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:57: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/Users/amsaha/gospel/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:39: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/Users/amsaha/gospel/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:76: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


In [4]:
import numpy as np
from numpy.random import PCG64, Generator
from typing import List, Dict

# Assuming these classes and functions are imported properly
# from your library
# from your_library import Secrets, Client, get_bipartite_coloring, TrappifiedCanvas, StimBackend, DepolarisingNoiseModel, generate_random_pauli_pattern, command

# Initialization
fx_bg = PCG64(42)
jumps = 5
n_iterations = 1000  # Number of test iterations

# Define separate outcome tables for Canonical and Deviant
test_outcome_table_canonical: List[Dict] = []
test_outcome_table_deviant: List[Dict] = []

# Loop over Construction Orders
for order in (ConstructionOrder.Canonical, ConstructionOrder.Deviant):
    rng = Generator(fx_bg.jumped(jumps))  # Use the jumped rng
    pattern = generate_random_pauli_pattern(
        nqubits=2, nlayers=2, order=order, rng=rng
    )

    # Add measurement commands to the output nodes
    for onode in pattern.output_nodes:
        pattern.add(command.M(node=onode))

    # Initialize secrets and client
    secrets = Secrets(r=False, a=False, theta=False)
    client = Client(pattern=pattern, secrets=secrets)
    
    # Get bipartite coloring and create test runs
    colours = get_bipartite_coloring(pattern)
    test_runs = client.create_test_runs(manual_colouring=colours)

    # Define backend and noise model
    backend = StimBackend()
    noise_model = DepolarisingNoiseModel(entanglement_error_prob=0.1)

    n_failures = 0

    # Choose the correct outcome table based on order
    if order == ConstructionOrder.Canonical:
        test_outcome_table = test_outcome_table_canonical
    else:
        test_outcome_table = test_outcome_table_deviant

    print(f"Running {n_iterations} iterations for order: {order}", flush=True)

    # Run the test iterations
    for i in range(n_iterations):
        # Select a random test run
        run = TrappifiedCanvas(test_runs[rng.integers(len(test_runs))], rng=rng)

        # Delegate the test run to the client
        trap_outcomes = client.delegate_test_run(backend=backend, run=run, noise_model=noise_model)
        
        # Create a result dictionary (trap -> outcome)
        result = {
            tuple(trap): outcome for trap, outcome in zip(run.traps_list, trap_outcomes)
        }

        # Append the result to the appropriate test outcome table
        test_outcome_table.append(result)

        # Print pass/fail based on the sum of the trap outcomes
        if sum(trap_outcomes) != 0:
            n_failures += 1
            print(f"Iteration {i+1}: ❌ Failed trap round", flush=True)
        else:
            print(f"Iteration {i+1}: ✅ Trap round passed", flush=True)

    # Final report after completing the test rounds
    print(f"Final result for {order}: {n_failures}/{n_iterations} failed rounds", flush=True)
    print("-" * 50, flush=True)

    # Uncomment this line if you want to assert no failures occurred
    # assert n_failures == 0, f"Test failed: {n_failures} trap rounds detected noise."

# Now you have two separate outcome tables:
# test_outcome_table_canonical for Canonical order
# test_outcome_table_deviant for Deviant order


Running 1000 iterations for order: ConstructionOrder.Canonical
Iteration 1: ❌ Failed trap round
Iteration 2: ❌ Failed trap round
Iteration 3: ❌ Failed trap round
Iteration 4: ❌ Failed trap round
Iteration 5: ❌ Failed trap round
Iteration 6: ❌ Failed trap round
Iteration 7: ❌ Failed trap round
Iteration 8: ❌ Failed trap round
Iteration 9: ❌ Failed trap round
Iteration 10: ❌ Failed trap round
Iteration 11: ❌ Failed trap round
Iteration 12: ❌ Failed trap round
Iteration 13: ❌ Failed trap round
Iteration 14: ❌ Failed trap round
Iteration 15: ❌ Failed trap round
Iteration 16: ❌ Failed trap round
Iteration 17: ❌ Failed trap round
Iteration 18: ❌ Failed trap round
Iteration 19: ❌ Failed trap round
Iteration 20: ❌ Failed trap round
Iteration 21: ❌ Failed trap round
Iteration 22: ❌ Failed trap round
Iteration 23: ❌ Failed trap round
Iteration 24: ❌ Failed trap round
Iteration 25: ❌ Failed trap round
Iteration 26: ❌ Failed trap round
Iteration 27: ❌ Failed trap round
Iteration 28: ❌ Failed trap 

In [5]:
print(len(test_outcome_table_canonical))
occurences = {}
occurences_one = {}

for results in test_outcome_table_canonical:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba = {q: occurences_one[q] / occurences[q] for q in occurences}
print(failure_proba)

1000
{(1,): 0.5173824130879345, (2,): 0.48261758691206547, (5,): 0.46830265848670755, (6,): 0.5337423312883436, (9,): 0.4785276073619632, (10,): 0.48057259713701433, (13,): 0.46421267893660534, (14,): 0.5132924335378323, (17,): 0.5214723926380368, (0,): 0.512720156555773, (3,): 0.5009784735812133, (4,): 0.4931506849315068, (7,): 0.5166340508806262, (8,): 0.487279843444227, (11,): 0.46966731898238745, (12,): 0.5264187866927593, (15,): 0.49510763209393344, (16,): 0.4794520547945205}


In [6]:
print(len(test_outcome_table_deviant))
occurences = {}
occurences_one = {}

for results in test_outcome_table_deviant:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba = {q: occurences_one[q] / occurences[q] for q in occurences}
print(failure_proba)

1000
{(1,): 0.49079754601226994, (2,): 0.5603271983640081, (5,): 0.4703476482617587, (6,): 0.5296523517382413, (9,): 0.5071574642126789, (10,): 0.46830265848670755, (13,): 0.523517382413088, (14,): 0.49284253578732107, (17,): 0.4703476482617587, (0,): 0.4911937377690802, (3,): 0.487279843444227, (4,): 0.5146771037181996, (7,): 0.5088062622309197, (8,): 0.5048923679060665, (11,): 0.4911937377690802, (12,): 0.5009784735812133, (15,): 0.5362035225048923, (16,): 0.4774951076320939}


In [10]:
#Working on the equations now